In [2]:
#rnn실습1
#단어를 순차적으로 나열

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
#'apple!' : input_data => apple, output_data=pple!

In [5]:

input_str='apple'
label_str='pple!'
char_set=sorted(list(set(input_str+label_str)))
print(char_set)
char_set_size=len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


	1.	input_str + label_str:
	•	두 문자열을 합쳐 "applepple!"을 만듭니다.
	2.	set():
	•	합쳐진 문자열에서 중복된 문자를 제거하여 고유 문자 집합을 만듭니다.
	•	결과: {'a', 'p', 'l', 'e', '!'}
	3.	list():
	•	집합을 리스트로 변환합니다.
	•	결과: ['a', 'p', 'l', 'e', '!'] (순서는 집합의 특성상 보장되지 않음).
	4.	sorted():
	•	리스트를 알파벳 순서로 정렬합니다.
	•	결과: ['!', 'a', 'e', 'l', 'p']
	5.	print(char_set):
	•	결과를 출력합니다.

In [6]:
#모델을 만들기 위해 필요한 정보들
#input과output은 정해짐
input_size=5
#은닉(기준이 없으니 임의로)
hidden_size=5
output_size=5
learning_rate=0.1

In [8]:
#문자 그대로를 훈련이 아닌 벡터값을 훈련
#c=키,i=값,enumerate=char순서의 인덱스 i와 char_set을 c에 동시에 넣음
#[('!', 0), ('a', 1), ('e', 2), ('l', 3), ('p', 4)]
#char_to_index=문자를 숫자로 매핑하는 딕셔너리를 생성하는 코드
char_to_index=dict((c,i)for i,c in enumerate(char_set))
print(char_to_index)
# char_list=[c for c in char_set]
#원래구조
# char_list=[]
# for c in char_set:
#   char_set.append(c)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [9]:
#반대로 숫자 → 문자 변환:
#역매핑을 하는 이유:사용자가 문자를 입력하면 숫자로 새로운 텍스트를 생성->생성된 숫자 데이터를 문자로 사용자에게 반환
	# •	입력: "hello"
	# •	출력 인덱스: [7, 4, 11, 11, 14] (각각의 문자를 숫자로 표현한 결과)
	# •	역매핑을 사용해 문자로 변환: "hello"
index_to_char={}
for key, value in char_to_index.items():
  index_to_char[value]=key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [22]:
x_data=[char_to_index[c] for c in input_str]#apple
y_data=[char_to_index[c] for c in label_str]#pple!
#차원을 높여!
x_data=[x_data]
y_data=[y_data]

print(x_data)
print(y_data)


[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [23]:
#원핫인코딩
#딥러닝 모델은 문자나 범주 데이터를 이해하지 못하고, 숫자 데이터만 처리할 수 있습니다. 
#문자를 숫자로 변환하는 과정이 필요 원-핫 인코딩은 범주형 데이터를 숫자로 표현하면서 데이터 간의 관계를 만들지 않도록 하기 위해 사용
x_one_hot=[np.eye(char_set_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


1 → [0., 1., 0., 0., 0.] (‘a’)
4 → [0., 0., 0., 0., 1.] (‘p’)
4 → [0., 0., 0., 0., 1.] (‘p’)
3 → [0., 0., 0., 1., 0.] (‘l’)
2 → [0., 0., 1., 0., 0.] (‘e’)


In [24]:
# 원-핫 인코딩된 입력 데이터 x_one_hot을 FloatTensor로 변환
# FloatTensor: 실수형 텐서
X = torch.FloatTensor(x_one_hot)

# 출력 데이터 y_data를 LongTensor로 변환
# LongTensor: 정수형 텐서 (모델이 정답 레이블로 사용)
Y = torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
tensor([[4, 4, 3, 2, 0]])


In [25]:
#nn.RNN(input_size,hidden_size,output_size)
class Net(nn.Module):
  def __init__(self, input_size, hidden_size,output_size) :
    super(Net,self).__init__()
    self.RNN = nn.RNN(input_size, hidden_size, batch_first=True)
    #batch_first : 입력데이터의 배치의 차원
    #(배치크기,시퀀스길이,특징 수)
    #입력할때 (32,10,50)이 되면 batch_first를 true로 했기 때문에 배치의 크기는 32,시퀀스길이:10,특징 수:50
    self.fc=nn.Linear(hidden_size,output_size,bias=True)
  #실제 구현
  def forward(self,x):
    x,_status=self.RNN(x)
    x=self.fc(x)
    return x

	•	문장: "I love AI"
	•	단어 집합: ['I', 'love', 'AI']
	•	입력 데이터:[
  [1, 0, 0],  # "I"
  [0, 1, 0],  # "love"
  [0, 0, 1],  # "AI"
]
  •	배치 크기 = 1 (문장이 하나만 입력됨)
	•	시퀀스 길이 = 3 (문장이 3개의 단어로 구성)
	•	특징 수 = 3 (단어를 3차원 원-핫 벡터로 표현)

In [26]:
#모델
model=Net(input_size,hidden_size,output_size)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [29]:
output=model(X)
print(output)
print(output.size())

tensor([[[ 0.0713, -0.2969, -0.2880, -0.2293,  0.3663],
         [ 0.1286, -0.2932, -0.0973, -0.1547,  0.1787],
         [ 0.0791, -0.0493, -0.2580, -0.2127,  0.3308],
         [-0.2663,  0.1718, -0.6839, -0.1815,  0.7529],
         [ 0.1142, -0.3721, -0.2642, -0.3323,  0.3330]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 5, 5])


In [36]:
#위 결과에서 가장 큰 인덱스
result=output.data.numpy().argmax(axis=-1)
#squeeze=크기가 1인 차원을 제거 
print(np.squeeze(result))

[4 4 3 2 0]


In [37]:
for i in range(200):
  optimizer.zero_grad()
  output=model(X)
  loss=criterion(output.view(-1,input_size),Y.view(-1))
  loss.backward()
  optimizer.step()

  result=output.data.numpy().argmax(axis=-1)
  str_result=''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i,"loss:",loss.item(),"prediction:",result,str_result,'true:',y_data)

0 loss: 0.00028130909777246416 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
1 loss: 0.0002803319366648793 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
2 loss: 0.00027935474645346403 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
3 loss: 0.00027837755624204874 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
4 loss: 0.00027735266485251486 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
5 loss: 0.00027637547464109957 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
6 loss: 0.0002753982844296843 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
7 loss: 0.00027446873718872666 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
8 loss: 0.00027349154697731137 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
9 loss: 0.0002725382219068706 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
10 loss: 0.00027160864556208253 prediction: [[4 4 3 2 0]] pple! true: [[4, 4, 3, 2, 0]]
11 loss: 0.0002706552913878113 prediction: [[

In [38]:
#여러줄
sentence = """For the past 11 years, Straus has been House chair of the joint Transportation Committee. 
He also sits on a transportation funding task force charged with considering 
what the next era of transportation funding could look like as the notoriously 
under-resourced MBTA barrels toward a fiscal cliff."""

In [39]:
#문자셋으로 만들기
#중복제거
char_set2=list(set(sentence))
print(char_set2)

['t', ' ', 'M', 'a', 'y', 'e', 'B', 'C', 'x', 'd', ',', 'k', '.', 'w', 'o', 'b', 'S', 'i', 'm', '\n', 'g', 'p', '1', 'h', 'c', 'l', 'r', 'u', 'T', 'f', 'F', 'n', 'j', '-', 'A', 's', 'H']


In [40]:
char_dic={c:i for i,c in enumerate(char_set2)}
char_dic

{'t': 0,
 ' ': 1,
 'M': 2,
 'a': 3,
 'y': 4,
 'e': 5,
 'B': 6,
 'C': 7,
 'x': 8,
 'd': 9,
 ',': 10,
 'k': 11,
 '.': 12,
 'w': 13,
 'o': 14,
 'b': 15,
 'S': 16,
 'i': 17,
 'm': 18,
 '\n': 19,
 'g': 20,
 'p': 21,
 '1': 22,
 'h': 23,
 'c': 24,
 'l': 25,
 'r': 26,
 'u': 27,
 'T': 28,
 'f': 29,
 'F': 30,
 'n': 31,
 'j': 32,
 '-': 33,
 'A': 34,
 's': 35,
 'H': 36}

In [41]:
#크기구하기
char_dic_size=len(char_dic)
print(char_dic_size)

37


In [42]:
hidden_size=char_dic_size
#sequence_length:세트를 나눔 글자 10개씩 한세트를 만들어 훈련을 하겠다!
sequence_length=10
learning_rate=0.1

In [46]:
x_data=[]
y_data=[]

for i in range(0,len(sentence)-sequence_length):
  x_str=sentence[i:i+sequence_length]
  y_str=sentence[i+1:i+sequence_length]
  print(i,x_str,':',y_str)
  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

0 For the pa : or the pa
1 or the pas : r the pas
2 r the past :  the past
3  the past  : the past 
4 the past 1 : he past 1
5 he past 11 : e past 11
6 e past 11  :  past 11 
7  past 11 y : past 11 y
8 past 11 ye : ast 11 ye
9 ast 11 yea : st 11 yea
10 st 11 year : t 11 year
11 t 11 years :  11 years
12  11 years, : 11 years,
13 11 years,  : 1 years, 
14 1 years, S :  years, S
15  years, St : years, St
16 years, Str : ears, Str
17 ears, Stra : ars, Stra
18 ars, Strau : rs, Strau
19 rs, Straus : s, Straus
20 s, Straus  : , Straus 
21 , Straus h :  Straus h
22  Straus ha : Straus ha
23 Straus has : traus has
24 traus has  : raus has 
25 raus has b : aus has b
26 aus has be : us has be
27 us has bee : s has bee
28 s has been :  has been
29  has been  : has been 
30 has been H : as been H
31 as been Ho : s been Ho
32 s been Hou :  been Hou
33  been Hous : been Hous
34 been House : een House
35 een House  : en House 
36 en House c : n House c
37 n House ch :  House ch
38  House cha : House 

In [48]:
print(x_data)
print(y_data)

[[30, 14, 26, 1, 0, 23, 5, 1, 21, 3], [14, 26, 1, 0, 23, 5, 1, 21, 3, 35], [26, 1, 0, 23, 5, 1, 21, 3, 35, 0], [1, 0, 23, 5, 1, 21, 3, 35, 0, 1], [0, 23, 5, 1, 21, 3, 35, 0, 1, 22], [23, 5, 1, 21, 3, 35, 0, 1, 22, 22], [5, 1, 21, 3, 35, 0, 1, 22, 22, 1], [1, 21, 3, 35, 0, 1, 22, 22, 1, 4], [21, 3, 35, 0, 1, 22, 22, 1, 4, 5], [3, 35, 0, 1, 22, 22, 1, 4, 5, 3], [35, 0, 1, 22, 22, 1, 4, 5, 3, 26], [0, 1, 22, 22, 1, 4, 5, 3, 26, 35], [1, 22, 22, 1, 4, 5, 3, 26, 35, 10], [22, 22, 1, 4, 5, 3, 26, 35, 10, 1], [22, 1, 4, 5, 3, 26, 35, 10, 1, 16], [1, 4, 5, 3, 26, 35, 10, 1, 16, 0], [4, 5, 3, 26, 35, 10, 1, 16, 0, 26], [5, 3, 26, 35, 10, 1, 16, 0, 26, 3], [3, 26, 35, 10, 1, 16, 0, 26, 3, 27], [26, 35, 10, 1, 16, 0, 26, 3, 27, 35], [35, 10, 1, 16, 0, 26, 3, 27, 35, 1], [10, 1, 16, 0, 26, 3, 27, 35, 1, 23], [1, 16, 0, 26, 3, 27, 35, 1, 23, 3], [16, 0, 26, 3, 27, 35, 1, 23, 3, 35], [0, 26, 3, 27, 35, 1, 23, 3, 35, 1], [26, 3, 27, 35, 1, 23, 3, 35, 1, 15], [3, 27, 35, 1, 23, 3, 35, 1, 15, 5], [27, 

In [49]:
#원-핫 인코딩
x_one_hot2=[np.eye(char_dic_size)[x] for x in x_data]
X=torch.FloatTensor(x_one_hot2)
Y=torch.LongTensor(y_data)
print(X)
print(Y)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [51]:
print(Y)

tensor([[14, 26,  1,  ...,  1, 21,  3],
        [26,  1,  0,  ..., 21,  3, 35],
        [ 1,  0, 23,  ...,  3, 35,  0],
        ...,
        [17, 35, 24,  ..., 24, 25, 17],
        [35, 24,  3,  ..., 25, 17, 29],
        [24,  3, 25,  ..., 17, 29, 29]])


In [50]:
print(X.size(),Y.size())

torch.Size([290, 10, 37]) torch.Size([290, 9])


In [52]:
class Net(nn.Module):
  def __init__(self,input_dim,hidden_dim,layers):
    super(Net, self).__init__()
    self.rnn=nn.RNN(input_dim,hidden_dim,num_layers=layers,
                    batch_first=True)
    self.fc=nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x,_status=self.rnn(x)
    x=self.fc(x)
    return x

In [53]:
model=Net(char_dic_size,hidden_size,2)
 #층을 2개 쌓는다는 의미
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [54]:
outputs=model(X)
print(outputs.size())

torch.Size([290, 10, 37])


In [56]:
result=outputs.view(-1,char_dic_size)
print(result.size())

torch.Size([2900, 37])


In [ ]:
#훈련
for i in range(200):
  optimizer.zero_grad()
  outputs=model(X)
  loss=criterion(outputs.view(-1,char_dic_size),Y.view(-1))

  loss.backward()
  optimizer.step()

  results=outputs.argmax(dim=2)
  predict_str=""

  for j,result in enumerate(results):
    if j==0 :
      predict_str+=''.join([char_set2[t] for t in result])
      